# 推論スクリプト

学習済みモデルを使用してテキスト分類の推論を行います。

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/)

## 1. 環境設定

まず、GPUが有効になっているか確認します。

**設定方法**: メニュー → ランタイム → ランタイムのタイプを変更 → GPU を選択

In [ ]:
# GPU確認
import torch
print(f"GPU利用可能: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU名: {torch.cuda.get_device_name(0)}")

## 2. ライブラリのインストール

In [ ]:
!pip install -q transformers torch

## 3. ライブラリのインポート

In [ ]:
import argparse
import logging

import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

## 4. ModelInferenceクラスの定義

In [ ]:
class ModelInference:
    """
    モデル推論クラス

    学習済みモデルをロードし、テキストの分類予測を行います。
    単一テキストとバッチ処理の両方に対応しています。

    Attributes:
        device (str): 使用するデバイス（'cuda' または 'cpu'）
        tokenizer: テキストをトークンに変換するトークナイザー
        model: 予測に使用する分類モデル
    """

    def __init__(self, model_path: str, device: str = None):
        """
        Args:
            model_path: モデルのパス
            device: 使用するデバイス（cuda/cpu）
        """
        self.device = device or "cpu"  # CPUを使用
        logger.info(f"Loading model from {model_path}")
        logger.info(f"Using device: {self.device}")

        self.tokenizer = AutoTokenizer.from_pretrained(model_path)
        self.model = AutoModelForSequenceClassification.from_pretrained(model_path)
        self.model.to(self.device)
        self.model.eval()

        # 推論最適化: キャッシュを有効化
        if hasattr(self.model, "config"):
            self.model.config.use_cache = True

    def predict(self, text: str) -> dict:
        """
        テキストの予測を行う

        Args:
            text: 入力テキスト

        Returns:
            予測結果の辞書
        """
        # トークナイズ
        inputs = self.tokenizer(
            text, return_tensors="pt", truncation=True, max_length=512, padding=True
        )

        # デバイスに転送
        inputs = {key: value.to(self.device) for key, value in inputs.items()}

        # 推論
        with torch.no_grad():
            outputs = self.model(**inputs)

        # 結果の処理
        logits = outputs.logits
        probabilities = torch.softmax(logits, dim=-1)
        predicted_class = torch.argmax(probabilities, dim=-1).item()
        confidence = probabilities[0][predicted_class].item()

        return {
            "predicted_class": predicted_class,
            "confidence": confidence,
            "probabilities": probabilities[0].cpu().numpy().tolist(),
        }

    def predict_batch(self, texts: list) -> list:
        """
        複数テキストの予測を行う

        Args:
            texts: 入力テキストのリスト

        Returns:
            予測結果のリスト
        """
        # トークナイズ
        inputs = self.tokenizer(
            texts, return_tensors="pt", truncation=True, max_length=512, padding=True
        )

        # デバイスに転送
        inputs = {key: value.to(self.device) for key, value in inputs.items()}

        # 推論
        with torch.no_grad():
            outputs = self.model(**inputs)

        # 結果の処理
        logits = outputs.logits
        probabilities = torch.softmax(logits, dim=-1)
        predicted_classes = torch.argmax(probabilities, dim=-1).cpu().numpy().tolist()
        confidences = [
            probabilities[i][pred].item() for i, pred in enumerate(predicted_classes)
        ]

        results = []
        for i, (pred_class, confidence) in enumerate(
            zip(predicted_classes, confidences)
        ):
            results.append(
                {
                    "text": texts[i],
                    "predicted_class": pred_class,
                    "confidence": confidence,
                    "probabilities": probabilities[i].cpu().numpy().tolist(),
                }
            )

        return results

## 5. 使用例

In [ ]:
# 使用例
# モデルパスの指定（事前に学習済みモデルをアップロードまたはHugging Faceから）
# model_path = "./models/my-model"  # ローカルパス
model_path = "cardiffnlp/twitter-roberta-base-sentiment-latest"  # Hugging Faceモデル例

# 推論クラスの初期化
inference = ModelInference(model_path, device="cuda" if torch.cuda.is_available() else "cpu")

# 単一テキストの予測
text = "This is great!"
result = inference.predict(text)
print(f"Input: {text}")
print(f"Predicted class: {result['predicted_class']}")
print(f"Confidence: {result['confidence']:.4f}")

# バッチ予測
texts = ["This movie was fantastic!", "Terrible movie, waste of time."]
results = inference.predict_batch(texts)
for res in results:
    print(f"Text: {res['text']}")
    print(f"Prediction: {res['predicted_class']} (Confidence: {res['confidence']:.4f})")
    print("-" * 50)